# Assignment 4

In [29]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

# This part depends on read_gpickle, wich is deprecated, so it will not work

In [30]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

AttributeError: module 'networkx' has no attribute 'download'

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [22]:
def graph_identification():
    degree_distributions = []
    clustering = []
    small_paths = []
    for graph in P1_Graphs:
        degree = nx.degree_histogram(graph)
        degree_distributions.append(degree)
    # G1 and G4 are PA
        clusters = nx.average_clustering(graph)
        clustering.append(clusters)
    # G2 and G3 are probably SW_L
        path_len = nx.average_shortest_path_length(graph)
        small_paths.append(path_len)
    # G2 and G3 are SW_L and G5 is SW_H        
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()

NameError: name 'P1_Graphs' is not defined

In [23]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


NameError: name 'P1_Graphs' is not defined

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [24]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [25]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [26]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    
    df = pd.DataFrame(index=G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Degree'] = pd.Series(dict(G.degree()))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G))
    
    df_train = df[~pd.isnull(df['ManagementSalary'])]
    df_test = df[pd.isnull(df['ManagementSalary'])]
    
    RFC = RandomForestClassifier()
    X_train = df_train.drop('ManagementSalary', axis=1)
    y_train = df_train['ManagementSalary']
    RFC.fit(X_train, y_train)
    
    X_test = df_test.drop('ManagementSalary', axis=1)
    y_proba = RFC.predict_proba(X_test)[:,1]
    predictions = pd.Series(y_proba, index=df_test.index)
    
    return predictions


salary_predictions()

1      0.00
65     1.00
18     0.16
215    0.89
283    1.00
       ... 
691    0.00
788    0.00
944    0.00
798    0.00
808    0.00
Length: 252, dtype: float64

In [27]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [32]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [33]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    import sklearn as sk
    
    df = future_connections
    df['Preferencial attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['Common neighbours'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    df['Jaccard coeficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['Resource allocation index'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    
    df_test = df[df['Future Connection'].isnull()]
    
    df_train = df.dropna()
    X_train = df_train[['Preferencial attachment', 'Common neighbours', 'Jaccard coeficient', 'Resource allocation index']]
    y_train = df_train['Future Connection']
    
    RFC = RandomForestClassifier()
    RFC.fit(X_train, y_train)
    
    X_test = df_test.drop('Future Connection', axis=1)
    y_proba = RFC.predict_proba(X_test)[:,1]
    
    predictions = pd.Series(y_proba, index=df_test.index)

    
    return predictions



new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.013239
(20, 426)     0.640000
(50, 989)     0.014975
(942, 986)    0.010990
                ...   
(165, 923)    0.015124
(673, 755)    0.009443
(939, 940)    0.010990
(555, 905)    0.020245
(75, 101)     0.000000
Length: 122112, dtype: float64

In [34]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"
future_connections

,Future Connection,Preferencial attachment,Common neighbours,Jaccard coeficient,Resource allocation index
"(6, 840)",0.0,2070,9,0.073770,0.136721
"(4, 197)",0.0,3552,2,0.015504,0.008437
"(620, 979)",0.0,28,0,0.000000,0.000000
"(519, 872)",0.0,299,2,0.060606,0.039726
"(382, 423)",0.0,205,0,0.000000,0.000000
...,...,...,...,...,...
"(165, 923)",NaN,714,0,0.000000,0.000000
"(673, 755)",NaN,3,0,0.000000,0.000000
"(939, 940)",NaN,6,0,0.000000,0.000000
"(555, 905)",NaN,160,0,0.000000,0.000000
